<a href="https://colab.research.google.com/github/vikrammitra/prometheus/blob/main/AlgoTradingTest02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sourcing Alpaca Credentials

In [ ]:
credentials = open("/content/drive/MyDrive/Colab Notebooks/alpaca.txt","r")
lines = credentials.readlines()
consumer_key = lines[0].rstrip().replace("alpaca_key=","")
consumer_secret = lines[1].rstrip().replace("alpaca_secret=","")
#print(consumer_secret)


In [ ]:
!pip install alpaca-trade-api requests beautifulsoup4 transformers accelerate nvidia-ml-py3 accelerate bitsandbytes plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 16.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.7/323.7 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.1/175.1 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.2 MB/s eta 0:00

In [ ]:
import pandas as pd
from alpaca_trade_api import REST, Stream, TimeFrame,TimeFrameUnit
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

API_KEY = consumer_key
API_SECRET = consumer_secret

def fetch_news_summary(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        summary = soup.find('p').text
        return summary.strip()
    except Exception as e:
        print(f"Error fetching news summary: {e}")
        return None

def get_news_data(ALPACA_API_KEY,ALPACA_SECRET_KEY,ticker):
    # Get News for last 30 days
    today = datetime.today() - timedelta(1)
    yesterday = today - timedelta(31)
    today = today.strftime('%Y-%m-%d')
    yesterday = yesterday.strftime('%Y-%m-%d')

    # Initialize Alpaca API
    # Get the news data
    rest_client = REST(ALPACA_API_KEY, ALPACA_SECRET_KEY)
    news_list = rest_client.get_news(ticker,yesterday, today,limit=1000)
    rest_client2 = REST(ALPACA_API_KEY, ALPACA_SECRET_KEY)
    bars = rest_client2.get_bars(ticker, TimeFrame(59, TimeFrameUnit.Minute), yesterday,today, adjustment='raw').df
    bars['time'] = bars.index.strftime('%Y-%m-%d %H:%M')
    bars['Ticker'] = ticker

    # Convert the news data into a pandas DataFrame
    data = {
        'Title': [],
        'time': [],
        'Source': [],
        'Url': [],
        'Summary': [],
        'Ticker' : []
    }

    for news in news_list:
        data['Title'].append(news.headline)
        data['time'].append(news.created_at.strftime('%Y-%m-%d %H:%M'))
        data['Source'].append(news.source)
        data['Url'].append(news.url)
        data['Summary'].append(fetch_news_summary(news.url))
        data['Ticker'].append(ticker)

    df = pd.DataFrame(data)
    df["time"] = pd.to_datetime(df["time"])
    bars["time"] = pd.to_datetime(bars["time"])

    return df,bars

if __name__ == "__main__":
    news_df,bars_data = get_news_data(API_KEY,API_SECRET,"SPY")

In [ ]:
import plotly.graph_objects as go


# Extract the OHLCV data
ohlc_data = bars_data[['open', 'high', 'low', 'close', 'volume']]

# Create a candlestick plot
fig = go.Figure(data=[go.Candlestick(x=ohlc_data.index,
                open=ohlc_data['open'],
                high=ohlc_data['high'],
                low=ohlc_data['low'],
                close=ohlc_data['close'])])

# Customize the layout of the plot
fig.update_layout(title='S&P 500 Candlestick Chart',
                  xaxis_title='Date',
                  yaxis_title='Price',
                  xaxis_rangeslider_visible=False)

# Display the plot
fig.show()

In [ ]:
from transformers import pipeline,AutoModel, AutoTokenizer, AutoModelForSequenceClassification
import numpy as np
import torch
import bitsandbytes
import accelerate

/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning:

The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.



/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [ ]:

torch.cuda.is_available()

def news_sentiment_handler(news):

      device = torch.device("cuda")
      df_array = np.array(news[["Title"]])
      df_list = list(df_array[:,0])

      tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
      model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert").to("cuda")
      inputs = tokenizer(df_list, padding = True, truncation = True, return_tensors='pt').to(device) #tokenize text to be sent to model
      outputs = model(**inputs)
      predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

      model.config.id2label
      positive = predictions[:, 0].tolist()
      negative = predictions[:, 1].tolist()
      neutral = predictions[:, 2].tolist()
      table = {'Headline':df_list, "Positive":positive, "Negative":negative, "Neutral":neutral}
      sentiment_data = pd.DataFrame(table, columns = ["Headline", "Positive", "Negative", "Neutral"])

      return(sentiment_data)

sentiment_df = news_sentiment_handler(news_df.head(200))



In [ ]:
news_sentiment_df = news_df.merge(sentiment_df, left_on='Title', right_on='Headline')


In [ ]:
merged_dataframe = pd.merge_asof(bars_data,news_sentiment_df.sort_values('time'), on="time", by="Ticker",
                                 tolerance=pd.Timedelta("60m"))
merged_dataframe = merged_dataframe.dropna()


In [ ]:
merged_dataframe['return'] = np.log(merged_dataframe['close']/merged_dataframe['close'].shift(1))
merged_dataframe['target'] = np.where(merged_dataframe['return'] > 0, 1, 0)
merged_dataframe=merged_dataframe.dropna()

In [ ]:
merged_dataframe

,open,high,low,close,volume,trade_count,vwap,time,Ticker,Title,Source,Url,Summary,Headline,Positive,Negative,Neutral,return,target
10,279.2669,279.5400,277.57,277.6000,8743379,85962,278.862117,2023-07-07 17:43:00,TSLA,10 Consumer Discretionary Stocks With Whale Al...,benzinga,https://www.benzinga.com/markets/options/23/07...,This whale alert can help traders discover the...,10 Consumer Discretionary Stocks With Whale Al...,0.072046,0.044208,0.883746,-0.006328,0
28,269.3600,270.3600,268.42,268.8200,11507887,102468,269.271292,2023-07-10 18:29:00,TSLA,10 Consumer Discretionary Stocks With Whale Al...,benzinga,https://www.benzinga.com/markets/options/23/07...,This whale alert can help traders discover the...,10 Consumer Discretionary Stocks With Whale Al...,0.072046,0.044208,0.883746,-0.032139,0
45,267.6200,268.3900,266.37,268.1284,9168596,83454,267.431563,2023-07-11 18:05:00,TSLA,10 Consumer Discretionary Stocks With Whale Al...,benzinga,https://www.benzinga.com/markets/options/23/07...,This whale alert can help traders discover the...,10 Consumer Discretionary Stocks With Whale Al...,0.072046,0.044208,0.883746,-0.002576,0
62,273.8732,274.4400,273.07,273.9364,7250027,68508,273.812541,2023-07-12 17:41:00,TSLA,10 Consumer Discretionary Stocks Whale Activit...,benzinga,https://www.benzinga.com/markets/options/23/07...,This whale alert can help traders discover the...,10 Consumer Discretionary Stocks Whale Activit...,0.064118,0.118037,0.817846,0.021430,1
80,276.0250,279.0000,275.61,278.7700,17331973,151107,277.493927,2023-07-13 18:16:00,TSLA,10 Consumer Discretionary Stocks With Whale Al...,benzinga,https://www.benzinga.com/markets/options/23/07...,This whale alert can help traders discover the...,10 Consumer Discretionary Stocks With Whale Al...,0.072046,0.044208,0.883746,0.017491,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,258.1700,260.4400,258.01,258.7506,11104760,92506,259.299930,2023-08-04 15:22:00,TSLA,Apple On Track For The Worst Revenue Decline S...,benzinga,https://www.benzinga.com/23/08/33576415/apple-...,"To gain an edge, this is what you need to know...",Apple On Track For The Worst Revenue Decline S...,0.013126,0.942429,0.044445,-0.000017,0
354,258.7800,261.0400,258.74,260.0950,9228841,76304,260.202391,2023-08-04 16:21:00,TSLA,Tesla's Stock Top Stories: Tesla Faces Off Aga...,benzinga,https://www.benzinga.com/news/23/08/33577937/t...,Today's top stories for Tesla Inc. TSLA encomp...,Tesla's Stock Top Stories: Tesla Faces Off Aga...,0.063498,0.499053,0.437449,0.005182,1
355,260.1000,260.1931,256.06,256.0601,10264162,89528,258.418142,2023-08-04 17:20:00,TSLA,"Dan Niles Warns Of 10% Market Correction, End ...",benzinga,https://www.benzinga.com/analyst-ratings/analy...,The market is perched on the edge of a potenti...,"Dan Niles Warns Of 10% Market Correction, End ...",0.046633,0.576450,0.376917,-0.015635,0
356,256.0800,256.3500,253.77,254.6831,14126701,129048,255.117341,2023-08-04 18:19:00,TSLA,10 Consumer Discretionary Stocks Whale Activit...,benzinga,https://www.benzinga.com/markets/options/23/08...,This whale alert can help traders discover the...,10 Consumer Discretionary Stocks Whale Activit...,0.064118,0.118037,0.817846,-0.005392,0


In [ ]:
model_input = merged_dataframe.drop(['time','Url','Summary','Headline','Title','Ticker','Source'], axis=1)
X = model_input.drop(['target'],axis = 1)
Y = model_input.target

In [ ]:
from sklearn.model_selection import train_test_split

#splitting the data and building the training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1, stratify=Y)

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


def model_selection(X,Y):
 seed = 7
 models = []
 models.append(('LogisticRegression', LogisticRegression(random_state=seed)))
 models.append(('LinearDiscriminantAnalysis', LinearDiscriminantAnalysis()))
 models.append(('KNeighborsClassifier', KNeighborsClassifier()))
 models.append(('DecisionTreeClassifier', DecisionTreeClassifier()))
 models.append(('RandomForestClassifier', RandomForestClassifier()))
 models.append(('ExtraTreesClassifier',ExtraTreesClassifier(random_state=seed)))
 models.append(('AdaBoostClassifier',AdaBoostClassifier(DecisionTreeClassifier(random_state=seed),random_state=seed,learning_rate=0.1)))
 models.append(('SVM',svm.SVC(random_state=seed)))
 models.append(('GradientBoostingClassifier',GradientBoostingClassifier(random_state=seed)))
 models.append(('MLPClassifier',MLPClassifier(random_state=seed)))
# evaluate each model in turn
 results = []
 names = []
 scoring = 'accuracy'
 for name, model in models:
  kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
  cv_results = cross_val_score(model, X, Y, cv=kfold, scoring=scoring)
  results.append(cv_results)
  names.append(name)
  msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
  print(msg)
 return results, names

results, names = model_selection(X_train,y_train)

clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
                                 max_depth=1, random_state=0).fit(X_train, y_train)
clf.score(X_test, y_test)


LogisticRegression: 0.455357 (0.209355)
LinearDiscriminantAnalysis: 0.785714 (0.147470)
KNeighborsClassifier: 0.616071 (0.182501)
DecisionTreeClassifier: 1.000000 (0.000000)
RandomForestClassifier: 0.987500 (0.037500)
ExtraTreesClassifier: 0.796429 (0.112429)
AdaBoostClassifier: 1.000000 (0.000000)
SVM: 0.357143 (0.141512)
GradientBoostingClassifier: 1.000000 (0.000000)
MLPClassifier: 0.482143 (0.210229)


1.0